# Exercise: Exploratory Data Analysis with Pandas
## Instructions
1. Data Collection
    - Search and collect data on the following: (1993 - 2023)
        - Israel interest tares
        - US interest rates
        - USD to ILS exchange rates
    - For each dataset, create a pandas DataFrame with the following columns:
        - Year
        - Month
        - Current_Rate (Interest rate or exchange rate for that month)
        - Change_From_Last_Month (Difference in Current_Rate from the previous month)
        - Optional: You may include additional columns if you find them useful (e.g., Change_Percentage, Date).



In [2]:
import numpy as np
import pandas as pd

In [53]:
israel_interset = pd.read_csv('./israel_interest.csv', parse_dates=['DATE'])
us_interset = pd.read_csv('./us_interset.csv', parse_dates=['DATE'])
usd_ils_comparation = pd.read_csv('./conversion_ratio.csv', parse_dates=['DATE'])

dt_arr = [israel_interset, us_interset, usd_ils_comparation]

for i in range(len(dt_arr)):
    dt_arr[i]['Year'] = dt_arr[i]['DATE'].dt.year
    dt_arr[i]['Month'] = dt_arr[i]['DATE'].dt.month
    dt_arr[i] = dt_arr[i].rename(columns={'CURRENT_RATE': 'Current_Rate'})
    dt_arr[i]['Change_From_Last_Month'] = dt_arr[i]['Current_Rate'].diff()
    dt_arr[i] = dt_arr[i][['Year', 'Month', 'Current_Rate', 'Change_From_Last_Month']]
    dt_arr[i] = dt_arr[i][dt_arr[i]['Year'].between(1993, 2023)]

israel_interset = dt_arr[0]
us_interset = dt_arr[1]
usd_ils_comparation = dt_arr[2]

,Year,Month,Current_Rate,Change_From_Last_Month
379,2023,8,4.655301,0.059872
380,2023,9,4.639770,-0.015531
381,2023,10,4.551020,-0.088751
382,2023,11,4.523561,-0.027459
383,2023,12,4.488699,-0.034862


## Part A: Single DataFrame Analysis 
1. Display the first 5 rows of the US interest rates DataFrame.

2. Calculate the average Israeli interest rate over the period covered by your data.

3. Find the month and year when the USD to ILS exchange rate had the highest increase from the previous month.

4. Determine the total number of months where the US interest rate decreased compared to the previous month.

5. Add a new column to the Israeli interest rates DataFrame called Change_Percentage, representing the percentage change from the previous month.



In [48]:
# A - 1
us_interset.head()

In [72]:
# A - 2
israel_interset['Current_Rate'].mean()

In [60]:
# A - 3
usd_ils_comparation.loc[usd_ils_comparation['Change_From_Last_Month'].idxmax()]

In [69]:
# A - 4
us_interset[us_interset['Change_From_Last_Month'] < 0].shape[0]

123

In [81]:
# A - 5
israel_interset['Change_Percentage'] = israel_interset['Current_Rate'].pct_change() * 100

## Part B: Merging DataFrames
1. Merge the US and Israeli interest rates DataFrames on Year and Month.

2. In the merged DataFrame, create a new column Rate_Difference that shows the difference between the Israeli and US interest rates for each month.

3. Identify all months when the Israeli interest rate was at least 1% higher than the US interest rate - IH (Israeli Higher).
Identify all months when the US interest rate was at least 1% higher than the Israeli interest rate - UH (US Higher).
Identify all months when the interest rates were within 1% of each other - N (Near)
make a new column called Rate_Comparison with the following values: IH, UH, or N.

4. Merge the exchange rate DataFrame with the merged interest rates DataFrame.

5. After merging all three DataFrames, find the correlation between the Rate_Difference and the Current_Rate of the exchange rate.
hint: find the connection between the difference in interest rates and the exchange rates, use any method you think is appropriate.
provide an explanation of the results (in Hebrew) - at least 2 rows of text + a proof of the connection(some output) you found.


In [96]:
# B - 1
interest_merged_dt = pd.merge(us_interset, israel_interset, on=['Year', 'Month'], how='inner')
interest_merged_dt = interest_merged_dt.rename(columns={
    'Current_Rate_x': 'US_Current_Rate',
    'Change_From_Last_Month_x': 'US_Change_From_Last_Month',
    'Current_Rate_y': 'IS_Current_Rate',
    'Change_From_Last_Month_y': 'IS_Change_From_Last_Month',
    'Change_Percentage': 'IS_Change_Percentage'
})

In [100]:
# B - 2
interest_merged_dt['Rate_Difference'] =  interest_merged_dt['US_Current_Rate'] - interest_merged_dt['IS_Current_Rate']

In [106]:
# B - 3
interest_merged_dt['Rate_Difference'] = interest_merged_dt['IS_Current_Rate'] - interest_merged_dt['US_Current_Rate']

interest_merged_dt['Rate_Comparison'] = np.where(
    interest_merged_dt['Rate_Difference'] > 1, 'IH',
    np.where(interest_merged_dt['Rate_Difference'] < -1, 'UH', 'N')
)

In [111]:
# B - 4
main_merged_dt = pd.merge(interest_merged_dt, usd_ils_comparation, on=['Year', 'Month'], how='inner')
main_merged_dt = main_merged_dt.rename(columns={
    'Current_Rate': 'USD_ILS_Current_Rate',
    'Change_From_Last_Month': 'USD_ILS_Change_From_Last_Month'
})

main_merged_dt.head()

,Year,Month,US_Current_Rate,US_Change_From_Last_Month,IS_Current_Rate,IS_Change_From_Last_Month,IS_Change_Percentage,Rate_Difference,Rate_Comparison,USD_ILS_Current_Rate,USD_ILS_Change_From_Last_Month
0,1993,1,3.02,NaN,11.547143,0.657578,NaN,8.527143,IH,2.765,0.001
1,1993,2,3.03,0.01,12.325500,0.778357,6.740690,9.295500,IH,2.795,0.030
2,1993,3,3.07,0.04,11.845000,-0.480500,-3.898422,8.775000,IH,2.768,-0.027
3,1993,4,2.96,-0.11,11.925333,0.080333,0.678205,8.965333,IH,2.728,-0.040
4,1993,5,3.00,0.04,12.079524,0.154190,1.292966,9.079524,IH,2.729,0.001


In [121]:
# B - 5
correlation = main_merged_dt[['Rate_Difference', 'USD_ILS_Current_Rate']].corr().iloc[0, 1]
correlation
#-0.205 יצא לנו
#זה אומר שכשהריבית בישראל גובהה יותר מארה"ב שער הדולר-שקל ירד, אבל מכיוון שהתוצאה יחסית קרובה ל0 אז זה אומר שהקשר הוא לא וודאי

-0.20560996003882268

## Part C: Analyzing Relationships Between DataFrames
1. Analyze whether increases in US interest rates are associated with increases or decreases in the USD to ILS exchange rate in the same month.

2. Determine if changes in the Israeli interest rate have a stronger impact on the exchange rate than changes in the US interest rate.

3. Find any time lags in the effect of interest rate changes on the exchange rate. For example, does a change in US interest rates affect the exchange rate in the following month?
(כלומר- האם כשיש שינוי בריבית כלשהי - האם מייד יש שינוי בשער ההמרה? או שיש דיליי של חודש/חודשים עד שמרגישים את ההשפעה - אם בכלל?)

4. Calculate the rolling past 3-month average of the exchange rate for each data time, and add it as a new column.

5. Identify any periods where both US and Israeli interest rates were decreasing, and analyze how the exchange rate behaved during those periods.



In [122]:
# 1
correlation = main_merged_dt['US_Current_Rate'].corr(main_merged_dt['USD_ILS_Current_Rate'])
correlation

-0.014360400900506942

In [125]:
#2

correlation_us_change = main_merged_dt['US_Change_From_Last_Month'].corr(main_merged_dt['USD_ILS_Change_From_Last_Month'])
correlation_israel_change = main_merged_dt['IS_Change_From_Last_Month'].corr(main_merged_dt['USD_ILS_Change_From_Last_Month'])

print(f"correlation_us_change: {correlation_us_change}")
print(f"correlation_israel_change: {correlation_israel_change}")

correlation_us_change: -0.04418886507363818
correlation_israel_change: -0.10173044446960712


In [127]:
#3
main_merged_dt['US_Change_Lag_1'] = main_merged_dt['US_Change_From_Last_Month'].shift(1)
main_merged_dt['IS_Change_Lag_1'] = main_merged_dt['IS_Change_From_Last_Month'].shift(1)
main_merged_dt['USD_ILS_Change_Lag_1'] = main_merged_dt['USD_ILS_Change_From_Last_Month'].shift(1)


correlation_us_change_lag = main_merged_dt['US_Change_Lag_1'].corr(main_merged_dt['USD_ILS_Change_From_Last_Month'])
correlation_israel_change_lag = main_merged_dt['IS_Change_Lag_1'].corr(main_merged_dt['USD_ILS_Change_From_Last_Month'])

print("correlation_us_change_lag: ", correlation_us_change_lag)
print("correlation_israel_change_lag: ", correlation_israel_change_lag)

correlation_us_change_lag:  0.03375291037857664
correlation_israel_change_lag:  -0.08475600468831979


In [136]:
#4
main_merged_dt['USD_ILS_Rolling_3Month_Avg'] = main_merged_dt['USD_ILS_Current_Rate'].rolling(window=3).mean()

# Bonus
choose 3 global / local events or periods that you think could have affected the exchange rate or interest rates
and analyze the effect of these events on the data you collected.
show proof of your analysis that supports your claim.

(כלומר- תבחנו אירועים שקרו בתקופה שאתם עובדים עליה ובדקו איך הם השפיעו על הנתונים שאתם עובדים איתם)
למשל: בחירות בארה״ב, בחירות בישראל, רוסיה אוקראינה, מלחמות בישראל וכו׳.

In [ ]:
# Your code here